In [14]:
import pandas as pd
import numpy as np
import datetime

In [2]:
auctions = pd.read_csv("auctions.csv", dtype={"ref_type_id": np.uint8, "source_id": np.uint8})

In [3]:
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [34]:
def create_window(df, initial_day, final_day, date_feature_name, id_feature_name, label_name):
    window = df.loc[(df[date_feature_name].dt.day >= initial_day) & (df[date_feature_name].dt.day <= final_day)]
    window = window.groupby(id_feature_name).agg({date_feature_name : 'min'}).reset_index()
    window[label_name] = (window[date_feature_name] - pd.to_datetime("2019-04-" + str(initial_day))).dt.total_seconds()
    return window

In [37]:
window_18to20 = create_window(auctions, 18, 20, "date", "device_id", "seconds_to_appear")

In [38]:
window_18to20.head()

,device_id,date,seconds_to_appear
0,41863526108385,2019-04-19 19:40:28.465866,157228.465866
1,135153013040192,2019-04-20 04:10:54.009137,187854.009137
2,161514654074162,2019-04-18 02:52:46.357746,10366.357746
3,181891380775191,2019-04-20 23:19:25.420614,256765.420614
4,186034136943920,2019-04-18 16:42:46.331894,60166.331894


## Ignorar esto (es un feature)

In [ ]:
auctions_sorted = auctions.sort_values(["device_id","date"], ascending = True)
auctions_sorted['timediff'] = np.where(auctions_sorted['device_id'] == auctions_sorted['device_id'].shift(), auctions_sorted['date'] - auctions_sorted['date'].shift(1), np.nan)
auctions_sorted['time_to_reappear'] = auctions_sorted['timediff'].dt.total_seconds()

In [10]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    return df_sorted